In [1]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from pydantic import BaseModel, ValidationError
from typing import Optional
import pandas as pd
import re
import time
import openai
import os

In [2]:
import pandas as pd

survey_result = pd.read_csv("survey_self_tracking.csv")

examples = []

for _, row in survey_result.iterrows():
    example_dict = {
        "Nama": row["Nama"],
        "Usia": row["Usia"],
        "Jenis_Kelamin": row["Jenis Kelamin"],
        "Asal": row["Asal"],
        "Penggunaan_Alat_Pelacak": row["Penggunaan Alat Pelacak"],
        "Jenis_Alat_Pelacak": row["Jenis Alat Pelacak"],
        "Motivasi_Real_Time_Data": row["Motivasi Real-Time Data (1-5)"],
        "Bandingkan_di_Media_Sosial": row["Bandingkan di Media Sosial (1-5)"],
        "Bantu_Konsistensi": row["Bantu Konsistensi (1-5)"],
        "Rasa_Bersalah_Jika_Tidak_Capai_Target": row["Rasa Bersalah Jika Tidak Capai Target (1-5)"],
        "Meningkatkan_Pola_Hidup": row["Meningkatkan Pola Hidup (1-5)"],
        "Menetapkan_Target_dari_Data": row["Menetapkan Target dari Data (1-5)"],
        "Manfaat_Terbesar": row["Manfaat Terbesar"],
        "Tantangan_Kekhawatiran": row["Tantangan atau Kekhawatiran"],
        "Fitur_Baru_Diinginkan": row["Fitur Baru yang Diinginkan"]
    }

    formatted_string = ', '.join([f"{k}: {v}" for k, v in example_dict.items()])
    examples.append({"example": formatted_string})

In [3]:
class SelfTrackingSurvey(BaseModel):
    Nama: str
    Usia: int
    Jenis_Kelamin: str
    Asal: str
    Penggunaan_Alat_Pelacak: str
    Jenis_Alat_Pelacak: str
    Motivasi_Real_Time_Data: int
    Bandingkan_di_Media_Sosial: int
    Bantu_Konsistensi: int
    Rasa_Bersalah_Jika_Tidak_Capai_Target: int
    Meningkatkan_Pola_Hidup: int
    Menetapkan_Target_dari_Data: int
    Manfaat_Terbesar: Optional[str] = None
    Tantangan_Kekhawatiran: Optional[str] = None
    Fitur_Baru_Diinginkan: Optional[str] = None


In [4]:
SYNTHETIC_FEW_SHOT_PREFIX = "contoh survei tentang penggunaan alat pelacak oleh pelari:\n\n"
SYNTHETIC_FEW_SHOT_SUFFIX = """buatlah satu entri survei baru berdasarkan contoh berikut:
Subjek: {subject}
Detail tambahan: {extra}

Pastikan:
- jangan mengulangi nama responden yang sudah ada.
- gunakan format yang sama dengan contoh.
- Nama responden harus unik dan sesuai dengan jenis kelamin.
- Gunakan variasi dalam usia, asal, dan motivasi lari.

Output hanya berisi SATU entri dan tidak lebih.
"""

OPENAI_TEMPLATE = PromptTemplate(
    input_variables=["example"], 
    template="{example}"
    )

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

subject = "generasi muda yang aktif berlari dan menggunakan alat pelacak untuk meningkatkan performa mereka."
extra = "mereka tertarik dengan fitur baru yang dapat membantu mereka menetapkan target berdasarkan data yang dikumpulkan."

In [ ]:
llm = ChatOpenAI(
  model = "qwen/qwen3-32b:free",
  temperature = 0.7,
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("QWEN_API_KEY"),
)

/var/folders/b9/5pxngp4x62q2qbmwnt1x80vr0000gn/T/ipykernel_1646/1305778179.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
synthetic_data = []
for i in range(50):
    prompt = prompt_template.format(subject=subject, extra=extra)
    response = llm.predict(prompt)

    print(f"\n[iterazione {i+1}] Output di dati sintetici:\n{response}")

    fields = [field.strip() for field in response.strip().split(",")]
    entry_dict = {}
    for field in fields:
        if ": " in field:
            key, value = field.split(": ", 1)
            entry_dict[key.strip()] = value.strip()

    try:
        validated = SelfTrackingSurvey(**entry_dict)
        synthetic_data.append(validated.model_dump())
    except ValidationError as e:
        print(f"[ERROR] Validation error on entry {i+1}:\n{e}")
    
    time.sleep(1.5)

/var/folders/b9/5pxngp4x62q2qbmwnt1x80vr0000gn/T/ipykernel_1646/1871316959.py:4: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)



[iterazione 1] Output di dati sintetici:
Nama: Dinda, Usia: 21, Jenis_Kelamin: Perempuan, Asal: Semarang, Penggunaan_Alat_Pelacak: Ya, setiap hari, Jenis_Alat_Pelacak: Smartwatch, Aplikasi smartphone, Motivasi_Real_Time_Data: 5, Bandingkan_di_Media_Sosial: 4, Bantu_Konsistensi: 5, Rasa_Bersalah_Jika_Tidak_Capai_Target: 3, Meningkatkan_Pola_Hidup: 5, Menetapkan_Target_dari_Data: 5, Manfaat_Terbesar: Dapat memantau progres latihan dengan akurat, Tantangan_Kekhawatiran: Kadang terlalu fokus pada target sehingga melupakan kenyamanan tubuh, Fitur_Baru_Diinginkan: Fitur penyesuaian target otomatis berdasarkan kondisi fisik harian

[iterazione 2] Output di dati sintetici:
Nama: Daniel, Usia: 21, Jenis_Kelamin: Laki-laki, Asal: Palembang, Penggunaan_Alat_Pelacak: Ya, setiap hari, Jenis_Alat_Pelacak: Smartwatch, Aplikasi khusus pelari (RunKeeper), Motivasi_Real_Time_Data: 5, Bandingkan_di_Media_Sosial: 3, Bantu_Konsistensi: 5, Rasa_Bersalah_Jika_Tidak_Capai_Target: 4, Meningkatkan_Pola_Hidup: 

In [7]:
df = pd.DataFrame(synthetic_data)
df

,Nama,Usia,Jenis_Kelamin,Asal,Penggunaan_Alat_Pelacak,Jenis_Alat_Pelacak,Motivasi_Real_Time_Data,Bandingkan_di_Media_Sosial,Bantu_Konsistensi,Rasa_Bersalah_Jika_Tidak_Capai_Target,Meningkatkan_Pola_Hidup,Menetapkan_Target_dari_Data,Manfaat_Terbesar,Tantangan_Kekhawatiran,Fitur_Baru_Diinginkan
0,Dinda,21,Perempuan,Semarang,Ya,Smartwatch,5,4,5,3,5,5,Dapat memantau progres latihan dengan akurat,Kadang terlalu fokus pada target sehingga melu...,Fitur penyesuaian target otomatis berdasarkan ...
1,Daniel,21,Laki-laki,Palembang,Ya,Smartwatch,5,3,5,4,5,5,Bisa tahu progres latihan dan tetap fokus pada...,Kadang terlalu fokus pada angka,Analisis pola pelari dengan rekomendasi jarak/...
2,Rizal,21,Laki-laki,Jakarta,Ya,Smartwatch,4,2,5,3,4,5,Bisa analisis progres lari mingguan,Overfitting data yang sulit dipahami,Fitur rekomendasi target otomatis berdasarkan ...
3,Dinda,21,Perempuan,Denpasar,Ya,Smartwatch,5,4,5,3,5,5,Bisa menyesuaikan target latihan berdasarkan d...,Data kadang tidak konsisten antar aplikasi,Fitur prediksi capaian berdasarkan tren data
4,Rizal,21,Laki-laki,Jakarta,Ya,Smartwatch,5,2,5,3,4,5,Meningkatkan performa lari melalui data akurasi,Kurangnya baterai tahan lama,Fitur analisis pola lari untuk rekomendasi lat...
5,Dinda,21,Perempuan,Semarang,Ya,Fitness tracker,5,4,5,4,5,5,Bisa menyesuaikan target berdasarkan kemampuan...,Kurangnya informasi tentang nutrisi yang relevan,Fitur rekomendasi target jarak dan tempo otoma...
6,Indah,21,Perempuan,Semarang,Ya,Smartwatch,4,4,5,4,5,5,Bisa menyesuaikan latihan berdasarkan data harian,Kurang fleksibel dalam menyesuaikan tujuan,Fitur rekomendasi target berdasarkan progres p...
7,Adrian,21,Laki-laki,Jakarta,Ya,Smartwatch,5,4,5,3,5,5,Memantau progres latihan secara akurat,Notifikasi terlalu sering mengganggu fokus saa...,Analisis pola lari otomatis untuk rekomendasi ...
8,Dinda,21,Perempuan,Denpasar,Ya,Smartwatch,5,3,4,3,5,5,Bisa memantau progres dan menyesuaikan latihan,Data terlalu banyak dan membingungkan,Fitur rekomendasi target berlari dinamis berda...
9,Dewi,23,Perempuan,Palembang,Ya,Smartwatch,4,3,5,4,5,5,Monitoring recovery time setelah lari,Data terlalu kompleks untuk dipahami,Rekomendasi target pribadi berdasarkan kondisi...
